<a href="https://colab.research.google.com/github/alexandrefigcam/algoritmo_genetico/blob/main/alg_genetico_descriptografia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando uma Cifra.




In [8]:
!pip install unidecode

In [9]:
#Importando bibliotecas

from unidecode import unidecode
import string
import numpy as np
import random
import matplotlib.pyplot as plt

## Primeiro passo: modelagem da linguagem.

Nesta etapa, será computada probabilidades para todos pares de bigramas bem coma para cada unigrma. Será construído um mapa qe será usando pelo algoritmo genético.

In [10]:
class Cypher:
    
    def __init__(self, alph=string.ascii_lowercase):
        self.alphabet = alph
        dim = len(alph)
        self.probUnigram = np.zeros(dim)
        self.probBigram  = np.ones((dim,dim))
    
    
    def computeMapping(self,filePath):
        fileObject = open(filePath, "r")        
        text = fileObject.read()
        fileObject.close()
        
        text = unidecode(text) # Retira caracteres especiais do texto.
        text = text.split()
        
        # Retira pontuação
        table = str.maketrans("", "", string.punctuation) 
        words = [w.translate(table) for w in text]
        
        # Retira valores numericos e palavras que tenham números.
        words = [w for w in words if w.isalpha()]
        
        # Transforma em letras minúsculas
        words = [w.lower() for w in words]
        
        # Computa probabilidades     
        for w in words:
            i= ord(w[0]) - 97 # a==97, "b"==98,...
            self.probUnigram[i] += 1
            for ch in w[1:]:
                j = ord(ch) -97
                self.probBigram[i,j] += 1
                i = j
        
        self.probUnigram = self.probUnigram/(self.probUnigram.sum())
        self.probBigram /=  (self.probBigram.sum(axis=1, keepdims=True))
        
        
        
            
    def computeLogProbability(self, criptedSentence, stringDecoder):
                
        # Cria decodificador
        decoder = {}
        for i in range(len(self.alphabet)):
            decoder[stringDecoder[i]] = self.alphabet[i]
        
        
        #print("Decoder",decoder)
        
        # Traduz sentença criptografada
        translatedStr = ""
        for c in criptedSentence:
            if c >='a' and c <='z':
                translatedStr += decoder[c]
            else:
                translatedStr += c

        
        
        translatedStr = translatedStr.split()
        
        # Computa a log probabilidade do novo decodificador
        logLikelyhood = 0.0
        
        for w in translatedStr:
            i = ord(w[0]) - 97
            logLikelyhood += np.log(self.probUnigram[i])
            for ch in w[1:]:
                j = ord(ch) - 97
                logLikelyhood += np.log(self.probBigram[i,j])
                i=j
        
        return logLikelyhood, translatedStr  
    
    
    def getAlphabet(self):
        return self.alphabet
    
    
    
    def __str__(self):
        result = ""
        dim = len(self.alphabet)
        
        result += "   "
        for ch in self.alphabet:
            result += (ch+"   ")
        
        result +="\n"
        
        for i in range(dim):
            result += (self.alphabet[i]+" ")
            for j in range(dim):
                result += "{:.2f} ".format(self.probBigram[i,j])
            
            result +="\n"
            
        return result




## Algoritmo genético.

* Nesta seção, ocorre a implementação do algoritmo genético. Cria-se decodificadores de strings, avaliando a logprobabilidade de cada um, que então é armazenada.A ideia é encontrar o melhor decodificados.

* O Algoritmo produz uma população de decodificadores e realiza o cruzamento entre os mais adaptados e, em seguida, testa os filhos gerados. A essência é encontrar o decodificador que melhor traduz o texto encriptado, analogamente ao que ocorre na seleção natural das espécies de Darwin.

In [11]:

#O Algoritmo genético recebe como parâmetros o tamanho da população, 
# o número de gerações, o tamanho do torneio,a probabilidade de cruzamento,
#a probabilidade de mutação, a dimensão do alfabeto, a mensagem a ser descriptografada, e o objeto
# da classe Cypher.
def geneticAlgorithm(pop_size, geracoes, tam_torneio, prob_cruzamento,prob_mut, dim , msg, cypher):
    
    pop = [] #Lista de indivíduos(cromossomos) pertencentes à população
    historico_log = [] # Lista de todas as log probabilidades
    bestDecoder = None #Melhor chave
    bestTranslate = None #Melhor tradução.


    #Gera cromossomos
    def gerar_cromossomo():

      letras = list(string.ascii_lowercase)
      cromossomo = []
      
      #Enquanto houver letras do alfabeto disponíveis
      # são selecionadas letras aleatória para compor um determinado cromossomo
      #(cada letra é um gene)
      while len(letras) > 0:
        letra = random.choice(letras)
        letras.remove(letra)
        cromossomo.append(letra)
      return cromossomo


    #Bloco que realiza a chamada da função para criar cromossomos
    # até que o tamanho da população seja atingido.
    for j in range(pop_size):
      pop.append(gerar_cromossomo())
      print(" Cromossomo gerado: ", pop[j])
    print(" ")

    #Processo de seleção de indivíduos mais aptos e reprodução.
    for i in range(geracoes):
      for j in range(tam_torneio):

        if random.random() < prob_cruzamento:

          pai1, pai2 = None, None
          
          #Seleciona, aleatoriamente, os índices dos cromossomos que
          # serão combinados para gerar cromossomos filhos.
          while True:
            pai1 = random.randint(0, pop_size -1)
            pai2 = random.randint(0, pop_size -1)
            if pai1 != pai2:
              break
            
          filho1, filho2 = [],[]
          #Criação da lista de genes ainda válidos para
          # inserção no cromossomo filho 1.
          gen1_validos =[]
          for i in range(ord('a'), ord('z')+1):
               gen1_validos.append(chr(i))
          #Criação da lista de genes ainda válidos para
          # inserção no cromossomo filho 2
          gen2_validos = []
          for i in range(ord('a'), ord('z')+1):
               gen2_validos.append(chr(i))

          
          while True:
            #Seleciona um ponto aletório para "cortar"
            # os cromossomos pais, tornando-se referência
            # para o crossing over
            ponto = random.randint(0, dim -1)
            if ponto > 0 and ponto < (dim - 1):
               
               #Filho 1 recebe a primeira parte do pai1 e o
               #filho 2 recebe a primeira parte do pai2
              for p in range(ponto):
                #Criação da primeira parcela do filho 1
                if pop[pai1][p] not in filho1:
                  filho1.append(pop[pai1][p])
                  gen1_validos.remove(pop[pai1][p])
                else:
                  e = random.choice(gen1_validos)
                  gen1_validos.remove(e)
                  filho1.append(e)
                #Criação da primeira parcela do filho 2
                if pop[pai2][p] not in filho2:
                  filho2.append(pop[pai2][p])
                  gen2_validos.remove(pop[pai2][p])
                else:
                  e = random.choice(gen2_validos)
                  gen2_validos.remove(e)
                  filho2.append(e)
              
              #Filho 1 recebe a segunda parte do pai 2
              # enquanto o filho 2 ecebe a segunda parte do pai 1.
              for p in range(ponto, dim):
                #Criação da segunda parcela do filho1
                if pop[pai2][p] not in filho1:
                  filho1.append(pop[pai2][p])
                  gen1_validos.remove(pop[pai2][p])
                else:
                  e = random.choice(gen1_validos)
                  gen1_validos.remove(e)
                  filho1.append(e)
                #Criação da segunda parcela do filho 2
                if pop[pai1][p] not in filho2:
                  filho2.append(pop[pai1][p])
                  gen2_validos.remove(pop[pai1][p])
                else:
                  e =random.choice(gen2_validos)
                  gen2_validos.remove(e)
                  filho2.append(e)


              #print(ponto)
              #print(" Pai1 : ", pop[pai1])
              #print(" Pai2 : ", pop[pai2])
              #print("Filho1: ", filho1)
              #print("Filho2: ",filho2)

              break

          #Aplicação da operação de mutação.
          if random.random() <= prob_mut:
            gene1, gene2 = None, None

            while True:
              gene1 = random.randint(0, dim -1)
              gene2 = random.randint(0, dim - 1)
              if gene1 != gene2:
                filho1[gene1], filho1[gene2] = filho1[gene2], filho1[gene1]
                filho2[gene1], filho2[gene2] = filho2[gene2], filho2[gene1]
                break

         
          #Avaliação do fitness para que, caso os filhos sejam mais aptos, substituam
          # seus pais, contribuindo para uma geração mais apta.
          (fitness_pai1, translatedP1) = cypher.computeLogProbability(msg, pop[pai1])
          (fitness_pai2, translatedP2) = cypher.computeLogProbability(msg, pop[pai2])
          (fitness_filho1, translatedF1) = cypher.computeLogProbability(msg, filho1)
          (fitness_filho2, translatedF2) = cypher.computeLogProbability(msg, filho2)

          if fitness_filho1 > fitness_pai1 or fitness_filho1 > fitness_pai2:
             if fitness_filho1 > fitness_pai1:
               pop.pop(pai1)
             else:
               pop.pop(pai2)
             pop.append(filho1)
          if fitness_filho2 > fitness_pai1 or fitness_filho2 > fitness_pai2:
             if fitness_filho2 > fitness_pai1:
               pop.pop(pai1)
             else:
               pop.pop(pai2)
             pop.append(filho2)

          bestDecoder = pop[0][:]
          for ind in range(1, pop_size):
            fitness, translated = cypher.computeLogProbability( msg, pop[ind])
            bestFitness, bestTranslate = cypher.computeLogProbability(msg, bestDecoder)
            if fitness > bestFitness:
               bestDecoder = pop[ind][:]

          
          bestFitness, bestTranslate = cypher.computeLogProbability(msg, bestDecoder)
          print(" Melhor decodificador: ", bestDecoder)
          print(" Tradução: ", bestTranslate)
          print("logProbabilidade: ", bestFitness)
          print("\n")

          historico_log.append(bestFitness)

    #Plota o histórico de log probabilidades
    plt.plot(historico_log)
    plt.show()

    print("Best decoder", bestDecoder)
    

    print("Best Log Likelihood",historico_log[-1] )
    bestTranslation = " ".join(bestTranslate)
    print("Best translation", bestTranslation)
         
          


  

  

## Create the main function

In [15]:
# Guia para execucao do algoritmo:
# A mensagem original possui Log likelihood de -932.0621606082733

message = "u vxhm sqwmahf fqtm vxh ivrhhv nmf cqwmf ni u hgphyvhf \
vxnv vxhrh tni n lhti um n snmh txuyx rwmi fqtm kz qmh tnss qc vxh \
anrfhm u shmv vxh qivshri n xnmf um rwkkuma fqtm vxhur xqrihi nmf \
rhyhubhf um hgyxnmah vtqphmyh n asnii qc xnsc nmf xnsc vtq cussi qc \
ixna vqknyyq nmf ni lwyx umcqrlnvuqm ni u yqwsf fhiurh nkqwv luii nfshr \
vq inz mqvxuma qc xnsc n fqdhm qvxhr phqpsh um vxh mhuaxkqwrxqqf um \
txql u tni mqv um vxh shniv umvhrhivhf kwv txqih kuqarnpxuhi u tni \
yqlphsshf vq suivhm vq"

cObject = Cypher()
cObject.computeMapping("corpus.txt")

#Execução do algoritmo.

-melhor chave encontrada: ['n', 'k', 'y', 'f', 'h', 'c', 'a', 'x', 'u', 'j', 'd', 's', 'l', 'm', 'q', 'p', 'o', 'r', 'i', 'v', 'w', 'b', 't', 'g', 'z', 'e']

-Melhor LogProbabilidade:-928.6488019639512

- A tradução:
  i then lounged down the street and found as i expected that there was a mews in a lane which runs down by one wall of the garden i lent the ostlers a hand in rubbing down their horses and received in exchange twopence a glass of half and half two fills of shag tobacco and as much information as i could desire about miss adler to say nothing of half a doken other people in the neighbourhood in whom i was not in the least interested but whose biographies i was compelled to listen to

In [ ]:

geneticAlgorithm( 10, 100, 100, 0.7, 0.5,  26, message, cObject)

In [ ]:


#plt.plot(historico_log)
#plt.show()

#print("Best decoder", bestDecoder)
# Print differences between original and obtained decoder
#print("Alphabet", "original", "optimal", "Equal?")
#for i in range(len(originalAlph)):
    #print(originalAlph[i], criptedAlph[i], bestDecoder[i], criptedAlph[i]==bestDecoder[i])
    

#print("Best Log Likelihood",historico_log[-1] )
#bestTranslation = " ".join(bestTranslate)
#print("Best translation", bestTranslate)